<a href="https://colab.research.google.com/github/devil-of-silicon-valley/cmpe255-project/blob/shabab/Baseline/Program_Baseline_Shabab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Group 5:  Reuter_50_50 Data Set (https://archive.ics.uci.edu/ml/datasets/Reuter_50_50Links to an external site.)

Identify the author of an article based on attributes describing their writing style

**GOAL: Identify the author of an article based on attributes describing their writing style**

In [5]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
# import libaries
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from glob import glob
import json

In [7]:
# setup debug for prints troubleshooting
debug = True

In [41]:
def load_json(json_file):
  with open(json_file) as f:
    data = json.load(f)
  return data

def write_json(json_file, data):
  with open(json_file, "w") as outfile:
    json.dump(data, outfile)

In [12]:
# get the train and set directories
train_dir = '/content/drive/MyDrive/Assignments/03: FA24: CMPE-255 Sec 33 - Data Mining/Project/C50train'
test_dir = '/content/drive/MyDrive/Assignments/03: FA24: CMPE-255 Sec 33 - Data Mining/Project/C50test'

In [32]:
# create a function to load the data from the directories
def load_data(dir, json_save_dir, debug=False):
  data_dict = {}
  # get the list of author directories
  author_dirs = sorted(glob(f"{dir}/*"))

  # now parse the directories for the author names and their associated texts
  for author_dir in author_dirs:
    # get the author name from the directory
    author_name = author_dir.split('/')[-1]

    if debug: print(f"Loading files for {author_name}")

    author_file_entry = {}
    for file_entry in sorted(glob(f"{author_dir}/*.txt")):
      # get te filename (ex: 2537newsML.txt)
      author_filename = file_entry.split('/')[-1]
      # Read the file and remove any new lines or raw strings
      author_file = open(file_entry, "r").read().replace('\n', '').replace('\r', '')
      # create the dictionary accordingly
      author_file_entry[author_filename] = author_file

    if debug: print(f"Adding {author_name} to data dictionary")

    # add the dictionary entry for the current author and their files
    data_dict[author_name] = author_file_entry

  write_json(json_save_dir, data_dict)

  return data_dict

In [33]:
train_json_save_dir = '/content/drive/MyDrive/Assignments/03: FA24: CMPE-255 Sec 33 - Data Mining/Project/C50train.json'
test_json_save_dir = '/content/drive/MyDrive/Assignments/03: FA24: CMPE-255 Sec 33 - Data Mining/Project/C50test.json'
train_data = load_data(dir=train_dir, json_save_dir=train_json_save_dir)
test_data = load_data(dir=test_dir, json_save_dir=test_json_save_dir)

In [ ]:
for key, val in train_data.items():

In [39]:
# convert to a data frame
train_df = pd.DataFrame.from_dict({(i,j): train_data[i][j]
                           for i in train_data.keys()
                           for j in train_data[i].keys()},
                       orient='index')
train_df

,0
"(AaronPressman, 106247newsML.txt)",The Internet may be overflowing with new techn...
"(AaronPressman, 120600newsML.txt)",The U.S. Postal Service announced Wednesday a ...
"(AaronPressman, 120683newsML.txt)",Elementary school students with access to the ...
"(AaronPressman, 136958newsML.txt)",An influential Internet organisation has backe...
"(AaronPressman, 137498newsML.txt)",An influential Internet organisation has backe...
...,...
"(WilliamKazer, 28223newsML.txt)",China's central bank chief has said that infla...
"(WilliamKazer, 282935newsML.txt)","China ushered in 1997, a year it has hailed as..."
"(WilliamKazer, 287736newsML.txt)",China issued tough new rules on the handling o...
"(WilliamKazer, 289747newsML.txt)",China will avoid bold moves in tackling its ai...


In [36]:
train_df.head()

,0
"(AaronPressman, 106247newsML.txt)",The Internet may be overflowing with new techn...
"(AaronPressman, 120600newsML.txt)",The U.S. Postal Service announced Wednesday a ...
"(AaronPressman, 120683newsML.txt)",Elementary school students with access to the ...
"(AaronPressman, 136958newsML.txt)",An influential Internet organisation has backe...
"(AaronPressman, 137498newsML.txt)",An influential Internet organisation has backe...


In [ ]:
# get name of directories, authors (these will be the labels)
train_sub = [name for name in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, name))]
label_lst = np.copy(train_sub)

if debug:
    print(train_dir)
    print(label_lst)

# setup the initial empty variables
train = []
train_v = []
label = []

# load the input data from C50train directory and process it

# format will be something like this
# ===================================
# | Label         | Train           |
# ===================================
# | AaronPressman | 2537newsML.txt  |
# ===================================
# | AaronPressman | 14014newsML.txt |
# ===================================
# | ...           | ...             |
# ===================================
# | AlanCrosby    | 10306newsML.txt |
# ===================================
# | ...           | ...             |


auth_idx = 0

# go within the author directory to get list of the file names, this will be the training data
for i in train_sub:
    sub2_dir  = '../C50train/' + i
    train_sub2 = [name for name in os.listdir(sub2_dir) if os.path.isfile(os.path.join(sub2_dir, name))]

    #if debug:
    #    print(sub2_dir)
    #    print(train_sub2)

    # in each author file, save the author as the label and the text as its training data
    for j in train_sub2:
        sub3  = '../C50train/' + i + '/' + j

        with open(sub3, 'r') as file:
            data = file.read()
            data_no_nw = data.replace('\n', '').replace('\r', '')
            train.append(data_no_nw)

        # append author index as label
        label.append(auth_idx)

    # increment author index
    auth_idx = auth_idx + 1

        #if debug:
        #    print(sub3)

if debug:
    print(np.shape(train))
    print(np.shape(label))

    # bin count looking at label
    unused, idx = np.unique(label, return_counts=True)
    #print(unused)
    print(idx)

    print(train[0])
    print(label[0])
    #print(label)

../C50train
['AaronPressman' 'AlanCrosby' 'AlexanderSmith' 'BenjaminKangLim'
 'BernardHickey' 'BradDorfman' 'DarrenSchuettler' 'DavidLawder'
 'EdnaFernandes' 'EricAuchard' 'FumikoFujisaki' 'GrahamEarnshaw'
 'HeatherScoffield' 'JaneMacartney' 'JanLopatka' 'JimGilchrist' 'JoeOrtiz'
 'JohnMastrini' 'JonathanBirt' 'JoWinterbottom' 'KarlPenhaul' 'KeithWeir'
 'KevinDrawbaugh' 'KevinMorrison' 'KirstinRidley' 'KouroshKarimkhany'
 'LydiaZajc' "LynneO'Donnell" 'LynnleyBrowning' 'MarcelMichelson'
 'MarkBendeich' 'MartinWolk' 'MatthewBunce' 'MichaelConnor' 'MureDickie'
 'NickLouth' 'PatriciaCommins' 'PeterHumphrey' 'PierreTran' 'RobinSidel'
 'RogerFillion' 'SamuelPerry' 'SarahDavison' 'ScottHillis' 'SimonCowell'
 'TanEeLyn' 'TheresePoletti' 'TimFarrand' 'ToddNissen' 'WilliamKazer']
(2500,)
(2500,)
[50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50
 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50
 50 50]
The Internet may be overflowing with new technolo

In [ ]:
# split the test data
X_train, X_test, y_train, y_test = train_test_split(train, label, test_size=0.1, random_state=1)

# vectorize x_train and x_test from text to matrix
vect = TfidfVectorizer(stop_words='english')
x_train_vec = vect.fit_transform(X_train)
x_test_vec = vect.transform(X_test)

# use logistical regression
model = LogisticRegression()
model.fit(x_train_vec, y_train)

# predict and check accuracy
pred = model.predict(x_test_vec)
rep = classification_report(y_test, pred)

if debug:
    print(rep)

              precision    recall  f1-score   support

           0       0.57      0.80      0.67         5
           1       1.00      1.00      1.00         5
           2       0.75      0.50      0.60         6
           3       0.40      0.67      0.50         3
           4       0.60      0.75      0.67         4
           5       1.00      0.71      0.83         7
           6       1.00      0.80      0.89         5
           7       0.50      0.50      0.50         2
           8       0.56      1.00      0.71         5
           9       1.00      0.40      0.57         5
          10       0.90      1.00      0.95         9
          11       0.88      0.88      0.88         8
          12       1.00      1.00      1.00         5
          13       0.70      1.00      0.82         7
          14       1.00      0.60      0.75         5
          15       1.00      1.00      1.00         6
          16       0.60      0.75      0.67         4
          17       0.60    

In [ ]:
# Load in C50test located ../C50test/
test_dir = '../C50test'
# get name of directories, authors (these will be the labels)
test_sub = [name for name in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir, name))]
test_lst = np.copy(train_sub)

if debug:
    print(test_dir)
    print(test_lst)

# setup the initial empty variables
test       = []
test_label = []

# load the input data from C50test directory and process it

auth_idx = 0

# go within the author directory to get list of the file names, this will be the training data
for i in train_sub:
    sub2_dir  = '../C50test/' + i
    test_sub2 = [name for name in os.listdir(sub2_dir) if os.path.isfile(os.path.join(sub2_dir, name))]

    #if debug:
    #    print(sub2_dir)
    #    print(train_sub2)

    # in each author file, save the text as its test data
    for j in test_sub2:
        sub3  = '../C50test/' + i + '/' + j

        with open(sub3, 'r') as file:
            data = file.read()
            data_no_nw = data.replace('\n', '').replace('\r', '')
            test.append(data_no_nw)

        test_label.append(auth_idx)

    auth_idx = auth_idx + 1

if debug:
    print(np.shape(test))

../C50test
['AaronPressman' 'AlanCrosby' 'AlexanderSmith' 'BenjaminKangLim'
 'BernardHickey' 'BradDorfman' 'DarrenSchuettler' 'DavidLawder'
 'EdnaFernandes' 'EricAuchard' 'FumikoFujisaki' 'GrahamEarnshaw'
 'HeatherScoffield' 'JaneMacartney' 'JanLopatka' 'JimGilchrist' 'JoeOrtiz'
 'JohnMastrini' 'JonathanBirt' 'JoWinterbottom' 'KarlPenhaul' 'KeithWeir'
 'KevinDrawbaugh' 'KevinMorrison' 'KirstinRidley' 'KouroshKarimkhany'
 'LydiaZajc' "LynneO'Donnell" 'LynnleyBrowning' 'MarcelMichelson'
 'MarkBendeich' 'MartinWolk' 'MatthewBunce' 'MichaelConnor' 'MureDickie'
 'NickLouth' 'PatriciaCommins' 'PeterHumphrey' 'PierreTran' 'RobinSidel'
 'RogerFillion' 'SamuelPerry' 'SarahDavison' 'ScottHillis' 'SimonCowell'
 'TanEeLyn' 'TheresePoletti' 'TimFarrand' 'ToddNissen' 'WilliamKazer']
(2500,)


In [ ]:
new_test_vec = vect.transform(test)

new_pred = model.predict(new_test_vec)

rep = classification_report(test_label, new_pred)

if debug:
    print(new_pred[0:999])
    print(rep)

[ 0  0 41  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0 44  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 41  0  0
  5  0  1  1  1  1 17  1 14 17  1 21 17 17  1 14 14  1 14 14  1  1  1 17
  1 17 17  1 17 17 17 17 17 17  1  1  1  1 14  1 14  1  1 17 17 17 17  1
  1  1  1  1  2 16  2 16 16 16 16 16 16 16  2 16  2 16 16 16 16 16 16 16
 16 16  2 16 16 16 16  2 44  2 44  2 16 16  0  2 16  2 16 16 16 16 16 16
 16 18  2  2 16  2 13 37  3 49 11 27 11 49  3 13  3  3 43  3 37 43 13 13
 13  3 49 13 13  3 37  3  3 49 49 49 43 43 13 13 34 34 34 34 34 34 34 34
 34 13 49 34  3 49 43 43  4  4  4  4  4  4  4  4  4 23  4  5  5 23 23 30
 23  4 30 23 30 23 23 23  4  4  4  4  4 23  4 23  4  4 23 23 23 23 23  4
  4  4  4 23 23 23 23 23 23 30  5  5  5  5  5  5 41  5  5  5  5  9  5  5
 39 39  5 22  5  5  5  5 22  5  5  5  5  9  5  5  5  5  5  5  5  5  5  5
  5  5  5  5  5  5 35  5  5 22  5  5 12  6  6  6  6 12  6  6  6 12  6 12
  6 12 12 12 12 12 12 12  6 12 12 12 12 12 12 12  6